In [ ]:
import requests
import pandas as pd
import time
import random

API_KEY = "d0fetb1r01qsv9eh4490d0fetb1r01qsv9eh449g"  # Replace with your actual API key

tickers = [
    "ADYEY",    # Adyen N.V.
    "AAGIY",    # AIA Group Ltd.
    "AIQUY",    # L'Air Liquide S.A.
    "EADSY",    # Airbus SE
    "BABA",     # Alibaba Group Holding Limited
    "GOOGL",    # Alphabet Inc.
    "AMZN",     # Amazon.com, Inc.
    "AAPL",     # Apple Inc.
    "ASML",     # ASML Holding N.V.
    "AVGO",     # Broadcom Inc.
    "BYDDY",    # BYD Company Ltd.
    "JD",       # JD.com, Inc.
    "MPNGY",    # Meituan
    "MSFT",     # Microsoft Corporation
    "NVDA",     # NVIDIA Corporation
    "PDD",      # PDD Holdings Inc.
    "PNGAY",    # Ping An Insurance
    "TCEHY",    # Tencent Holdings Ltd.
    "TSM",      # Taiwan Semiconductor Manufacturing Company
    "TCOM"      # Trip.com Group Limited
]

rows = []

def fetch_data(ticker):
    base_url = "https://finnhub.io/api/v1"

    row = {
        "Ticker": ticker,
        "Company": "N/A",
        "PE Ratio": "N/A",
        "Profit Margin": "N/A",
        "Mean Rating": "N/A",
        "Recommendation": "N/A",
        "No. of Analysts": "N/A",
        "Market Cap": "N/A",
        "Country": "N/A",
        "Industry": "N/A"
    }

    try:
        # Company profile
        profile = requests.get(f"{base_url}/stock/profile2?symbol={ticker}&token={API_KEY}").json()
        row["Company"] = profile.get("name", "N/A")
        row["Country"] = profile.get("country", "N/A")
        row["Industry"] = profile.get("finnhubIndustry", "N/A")
        mc = profile.get("marketCapitalization")
        if mc is not None:
            if mc >= 1_000_000:
                row["Market Cap"] = f"${mc / 1_000_000:.1f}T"
            else:
                row["Market Cap"] = f"${mc:.1f}B"

        # Financial metrics
        metrics = requests.get(f"{base_url}/stock/metric?symbol={ticker}&metric=all&token={API_KEY}").json().get("metric", {})
        pe = metrics.get("peNormalizedAnnual")
        if pe: row["PE Ratio"] = round(pe, 2)
        margin = metrics.get("netProfitMarginAnnual")
        if margin: row["Profit Margin"] = f"{round(margin, 2)}%"

        # Analyst recommendations
        recs = requests.get(f"{base_url}/stock/recommendation?symbol={ticker}&token={API_KEY}").json()
        if recs:
            latest = recs[0]
            counts = {
                "Strong Buy": latest.get("strongBuy", 0),
                "Buy": latest.get("buy", 0),
                "Hold": latest.get("hold", 0),
                "Sell": latest.get("sell", 0),
                "Strong Sell": latest.get("strongSell", 0)
            }
            total = sum(counts.values())
            row["No. of Analysts"] = total
            if total > 0:
                rating = (
                    counts["Strong Buy"] * 1 +
                    counts["Buy"] * 2 +
                    counts["Hold"] * 3 +
                    counts["Sell"] * 4 +
                    counts["Strong Sell"] * 5
                ) / total
                row["Mean Rating"] = round(rating, 2)
                row["Recommendation"] = (
                    "Buy" if rating <= 2 else
                    "Hold" if rating <= 3 else
                    "Sell"
                )

    except Exception as e:
        print(f"⚠️ Error with {ticker}: {e}")

    return row

# Loop through tickers
for ticker in tickers:
    print(f"Fetching {ticker}...")
    rows.append(fetch_data(ticker))
    time.sleep(random.uniform(1.2, 1.8))

# Create DataFrame
df = pd.DataFrame(rows)

# Export to CSV
df.to_csv("finnhub_stock_data.csv", index=False)

df


Fetching ADYEY...
Fetching AAGIY...
Fetching AIQUY...
Fetching EADSY...
Fetching BABA...
Fetching GOOGL...
Fetching AMZN...
Fetching AAPL...
Fetching ASML...
Fetching AVGO...
Fetching BYDDY...
Fetching JD...
Fetching MPNGY...
Fetching MSFT...
Fetching NVDA...
Fetching PDD...
Fetching PNGAY...
Fetching TCEHY...
Fetching TSM...
Fetching TCOM...


,Ticker,Company,PE Ratio,Profit Margin,Mean Rating,Recommendation,No. of Analysts,Market Cap,Country,Industry
0,ADYEY,Adyen NV,53.19,41.07%,2.02,Hold,43,$49206.5B,NL,Financial Services
1,AAGIY,AIA Group Ltd,95.39,30.14%,1.59,Buy,29,$652100.6B,HK,Insurance
2,AIQUY,L'Air Liquide Societe Anonyme pour l'Etude et ...,31.90,12.22%,1.97,Buy,32,$105466.6B,FR,Chemicals
3,EADSY,Airbus SE,29.62,6.11%,1.96,Buy,25,$125358.2B,NL,Aerospace & Defense
4,BABA,Alibaba Group Holding Ltd,3.74,8.47%,1.69,Buy,49,$298219.3B,HK,Retail
5,GOOGL,Alphabet Inc,18.59,28.6%,1.95,Buy,64,$1.9T,US,Media
6,AMZN,Amazon.com Inc,34.74,9.29%,1.80,Buy,79,$2.0T,US,Retail
7,AAPL,Apple Inc,31.62,23.97%,2.11,Hold,56,$3.0T,US,Technology
8,ASML,ASML Holding NV,32.28,26.79%,2.00,Buy,42,$244424.6B,NL,Semiconductors
9,AVGO,Broadcom Inc,165.83,11.43%,1.83,Buy,46,$978945.7B,US,Semiconductors
